In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import LongType, StringType, BooleanType, TimestampType, IntegerType, ShortType
from delta.tables import DeltaTable
from datetime import datetime
from pyspark.sql.functions import col, regexp_replace, sha2, when, lit, uuid

CATALOGO_ORIGEM = "v_credit"
SCHEMA_ORIGEM = "bronze"
TABELA_ORIGEM = "chamados"

CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_chamado"
TABELA_INVALIDOS_DESTINO = "tb_chamado_invalidos"

nome_tabela_origem = f"{CATALOGO_ORIGEM}.{SCHEMA_ORIGEM}.{TABELA_ORIGEM}"
nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"

timestamp_atual = F.current_timestamp()


In [0]:
df_chamado = spark.table("v_credit.bronze.chamados")

chamado_limpo = (
    df_chamado
    .withColumnRenamed("id_chamado", "cd_chamado")
    .withColumnRenamed("id_cliente", "cd_cliente")
    .withColumnRenamed("motivo", "ds_motivo")
    .withColumnRenamed("canal", "cd_canal")
    .withColumnRenamed("resolvido", "st_resolvido")
    .withColumnRenamed("tempo_espera", "tm_espera")
    .withColumnRenamed("tempo_atendimento", "tm_duracao")
    .withColumnRenamed("id_atendente", "cd_atendente")
    .withColumnRenamed("ingestion_timestamp", "dt_ingestion")
    .withColumnRenamed("origem", "dc_origem")

    .drop("hora_abertura_chamado")
    .drop("hora_inicio_atendimento")
    .drop("hora_finalizacao_atendimento")
    .drop("ctid_fivetran_id")
    .drop("_fivetran_deleted")
    .drop("_fivetran_synced")
    
    .withColumn("cd_cliente", F.col("cd_cliente").cast(StringType()))
    .withColumn("ds_motivo", F.col("ds_motivo").cast(StringType()))
    
    .withColumn("dt_ingestion", F.col("dt_ingestion").cast(TimestampType()))
    .withColumn("dc_origem", F.col("dc_origem").cast(StringType()))
)

chamado_limpo.printSchema()

In [0]:
chamado_limpo = chamado_limpo.withColumn(
    "st_resolvido",
    regexp_replace(F.col("st_resolvido"), "(?i)n.o", "false")
).withColumn(
    "st_resolvido",
    regexp_replace(F.col("st_resolvido"), "(?i)sim", "true")
).withColumn(
    "st_resolvido",
    F.col("st_resolvido").cast(BooleanType())
)

chamado_limpo = chamado_limpo.withColumn("cd_cliente",  F.sha2(F.col("cd_cliente").cast(StringType()), 256))

In [0]:
df_log = spark.table("v_credit.silver.tb_chamado_log")

chamado_limpo = chamado_limpo.join(
    df_log.select("cd_chamado", "dh_abertura", "dh_inicio", "dh_fim"),
    on="cd_chamado",
    how="left"
)

chamado_limpo = (
    chamado_limpo
    .withColumn("tm_espera", (F.col("dh_inicio").cast("long") - F.col("dh_abertura").cast("long")))
    .withColumn("tm_duracao", (F.col("dh_fim").cast("long") - F.col("dh_inicio").cast("long")))
    .withColumn("tm_espera", F.col("tm_espera").cast(LongType()))
    .withColumn("tm_duracao", F.col("tm_duracao").cast(LongType()))
)

display(chamado_limpo)

In [0]:
from pyspark.sql import functions as F

mapa_correcoes = {
    "Bloqueio de cart.*o": "Bloqueio de cartão",
    "Contesta..o de fatura": "Contestação de fatura",
    "D.vidas gerais sobre programa de pontos": "Dúvidas gerais sobre programa de pontos",
    "Compra n.o autorizada": "Compra no autorizada", 
    "Contrata..o de cart.o adicional": "Contratação de cartão adicional",
    "Altera..o de dados cadastrais.*": "Alteração de dados cadastrais (vencimento da fatura, telefone, email)",
    "Desbloqueio de cart.o": "Desbloqueio de cartão",
    "Transfer.ncia de ag.ncia": "Transferência de agência",
    "Renegocia..o de d.vida": "Renegociação de dívida",
    "Problema com aplicativo": "Problema com aplicativo"
}

chamado_limpo_tratado = chamado_limpo
for padrao, correcao in mapa_correcoes.items():
    chamado_limpo_tratado = chamado_limpo_tratado.withColumn(
        "ds_motivo", 
        F.regexp_replace(F.col("ds_motivo"), padrao, correcao)
    )

chamado_limpo_tratado = chamado_limpo_tratado.withColumn(
    "ds_motivo", F.trim(F.col("ds_motivo"))
)

tb_motivo = spark.table("v_credit.silver.tb_motivo")

chamado_final = (
    chamado_limpo_tratado.alias("c")
    .join(
        tb_motivo.alias("m"),
        F.lower(F.col("c.ds_motivo")) == F.lower(F.trim(F.col("m.ds_motivo"))),
        "left"
    )
    .select(
        *[F.col(f"c.{col}") for col in chamado_limpo.columns if col != "ds_motivo"],
        
        F.col("m.cd_motivo").alias("ds_motivo")
    )
)

display(chamado_final.filter(F.col("ds_motivo").isNull()))

In [0]:
tb_canal = spark.table("v_credit.silver.tb_canal")

chamado_limpo = (
    chamado_final
    .withColumn("cd_canal", F.upper(F.trim(F.col("cd_canal"))))
    
    .withColumn("cd_canal", F.regexp_replace(F.col("cd_canal"), r"(?i)U\.?R\.?A\.?", "URA"))
    
    .withColumn("cd_canal", F.regexp_replace(F.col("cd_canal"), r"(?i)CHAT\s?BOT", "CHATBOT"))

    .withColumn("cd_canal", F.regexp_replace(F.col("cd_canal"), r"(?i)^ATEND.*INICIAL$", "ATENDIMENTO INICIAL"))
    
    .withColumn("cd_canal", F.regexp_replace(F.col("cd_canal"), r"(?i)^ATEND.*ESPECIALIZADO$", "ATENDIMENTO ESPECIALIZADO"))
)

chamado_validacao = (
    chamado_limpo.alias("c")
    .join(
        tb_canal.alias("can"),
        F.col("c.cd_canal") == F.col("can.nm_canal"),
        "left"
    )
    .select(
        *[F.col(f"c.{col}") for col in chamado_limpo.columns if col != "cd_canal"],
        
        F.col("can.cd_canal").alias("cd_canal")
    )
)

display(chamado_validacao)

In [0]:
df_final_merge = (
    chamado_validacao
    .withColumnRenamed("ds_motivo", "cd_motivo")
    .withColumn(
        "cd_atendente",
        F.when(F.col("cd_atendente") == "", None)
         .otherwise(F.col("cd_atendente"))
         .cast(LongType())
    )
)

display(df_final_merge)
df_final_merge.printSchema()

In [0]:
df_final_merge = (
    chamado_validacao
    .withColumnRenamed("ds_motivo", "cd_motivo")
    .withColumn("cd_atendente", F.expr("try_cast(cd_atendente AS BIGINT)"))
    .select(
        "cd_chamado",
        "cd_cliente",
        "cd_motivo",
        "cd_canal",
        "st_resolvido",
        "tm_espera",
        "tm_duracao",
        "cd_atendente",
        "dt_ingestion",
        "dc_origem"
    )
)

delta_table = DeltaTable.forName(spark, nome_tabela_destino)

(
    delta_table.alias("target")
    .merge(
        df_final_merge.alias("source"),
        "target.cd_chamado = source.cd_chamado"
    )
    .whenMatchedUpdateAll()
    .whenNotMatchedInsertAll()
    .execute()
)

print("Merge realizado com sucesso!")